In [1]:
import re
from functools import total_ordering

In [2]:
@total_ordering
class Version:
    def __init__(self, version):
        self.version = version
        self._parse_version(version)
    
    def _parse_version(self, version):
        version = version.lstrip('v')
        
        if '-' in version:
            main_version, pre_release = version.split('-', 1)
        else:
            main_version = version
            pre_release = None
        
        if pre_release is None:
            match = re.match(r'^(\d+\.\d+\.\d+)([a-zA-Z].*)$', main_version)
            if match:
                main_version = match.group(1)
                pre_release = match.group(2)
        
        version_parts = main_version.split('.')
        if len(version_parts) < 3:
            version_parts.extend(['0'] * (3 - len(version_parts)))
        
        try:
            self.major = int(version_parts[0])
            self.minor = int(version_parts[1])
            self.patch = int(version_parts[2])
        except (ValueError, IndexError):
            raise ValueError(f"Invalid version format: {version}")
        
        self.pre_release = pre_release
        self._parse_pre_release()
    
    def _parse_pre_release(self):
        """Parse pre-release identifiers for comparison"""
        if self.pre_release is None:
            self.pre_release_parts = None
            return
        
        parts = self.pre_release.split('.')
        parsed_parts = []
        
        for part in parts:
            numbers = re.findall(r'\d+', part)
            if numbers:
                parsed_parts.append((0, int(numbers[0]), part))
            else:
                parsed_parts.append((1, 0, part))
        
        self.pre_release_parts = parsed_parts
    
    def _compare_pre_release(self, other):
        """Compare pre-release versions according to semver rules"""
        if self.pre_release_parts is None and other.pre_release_parts is None:
            return 0
        if self.pre_release_parts is None:
            return 1
        if other.pre_release_parts is None:
            return -1
        
        for i in range(max(len(self.pre_release_parts), len(other.pre_release_parts))):
            if i >= len(self.pre_release_parts):
                return -1
            if i >= len(other.pre_release_parts):
                return 1
            
            self_part = self.pre_release_parts[i]
            other_part = other.pre_release_parts[i]
            
            if self_part[0] != other_part[0]:
                return self_part[0] - other_part[0]
            
            if self_part[0] == 0:
                if self_part[1] != other_part[1]:
                    return self_part[1] - other_part[1]
            
            if self_part[2] != other_part[2]:
                return -1 if self_part[2] < other_part[2] else 1
        
        return 0
    
    def __eq__(self, other):
        if not isinstance(other, Version):
            return False
        return (self.major == other.major and 
                self.minor == other.minor and 
                self.patch == other.patch and 
                self.pre_release == other.pre_release)
    
    def __lt__(self, other):
        if not isinstance(other, Version):
            return NotImplemented
        
        if self.major != other.major:
            return self.major < other.major
        if self.minor != other.minor:
            return self.minor < other.minor
        if self.patch != other.patch:
            return self.patch < other.patch
        
        pre_cmp = self._compare_pre_release(other)
        return pre_cmp < 0
    
    def __str__(self):
        return self.version
    
    def __repr__(self):
        return f"Version('{self.version}')"

In [5]:
def main():
    to_test = [
        ("1.0.0", "2.0.0"),
        ("1.0.0", "1.42.0"),
        ("1.2.0", "1.2.42"),
        ("1.1.0-alpha", "1.2.0-alpha.1"),
        ("1.0.1b", "1.0.10-alpha.beta"),
        ("1.0.0-rc.1", "1.0.0"),
    ]
    
    print("Testing version comparisons:")
    for left, right in to_test:
        left_v = Version(left)
        right_v = Version(right)
        print(f"{left} < {right}: {left_v < right_v}")
        print(f"{right} > {left}: {right_v > left_v}")
        print(f"{right} != {left}: {right_v != left_v}")
        print()
        
        assert Version(left) < Version(right), f"le failed"
        assert Version(right) > Version(left), f"ge failed"
        assert Version(right) != Version(left), f"neq failed"
    
    print("All tests passed!")

In [6]:
if __name__ == "__main__":
    main()

Testing version comparisons:
1.0.0 < 2.0.0: True
2.0.0 > 1.0.0: True
2.0.0 != 1.0.0: True

1.0.0 < 1.42.0: True
1.42.0 > 1.0.0: True
1.42.0 != 1.0.0: True

1.2.0 < 1.2.42: True
1.2.42 > 1.2.0: True
1.2.42 != 1.2.0: True

1.1.0-alpha < 1.2.0-alpha.1: True
1.2.0-alpha.1 > 1.1.0-alpha: True
1.2.0-alpha.1 != 1.1.0-alpha: True

1.0.1b < 1.0.10-alpha.beta: True
1.0.10-alpha.beta > 1.0.1b: True
1.0.10-alpha.beta != 1.0.1b: True

1.0.0-rc.1 < 1.0.0: True
1.0.0 > 1.0.0-rc.1: True
1.0.0 != 1.0.0-rc.1: True

All tests passed!
